<a href="https://colab.research.google.com/github/kapeeshvarma/Appliances-Energy-Prediction/blob/main/Appliances_Energy_Prediction_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**APPLIANCES ENERGY PREDICTION**

In this time of global uncertainty world needs energy and in increasing quantities to support economic and social progress and build a better quality of life, in particular in developing countries. But even today there are many regions which face constant power outages. These outages are primarily caused due to excess load consumed by home appliances.

Hence, the ability to predict energy consumption can not only save money for end user but can also help in generating money by giving excess energy back to Grid (in case of solar panels usage). In this project, we perform regression analysis in order to predict Appliance energy usage based on data collected from various sensors.

## Importing Data and Modules

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
